<a href="https://colab.research.google.com/github/akshala/POS-tagging-using-HMM-and-viterbi-algorithm/blob/main/hmm_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files   # upload Brown_train.txt
uploaded = files.upload()

Saving Brown_train.txt to Brown_train.txt


In [ ]:
from statistics import mean
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
from statistics import harmonic_mean

In [ ]:
with open('Brown_train.txt') as f:    # read file
  text = f.read()
# text

In [ ]:
sentences = text.split('\n')     # split into sentences
sentences = ['<s>_<s> ' + elt for elt in sentences if len(elt) > 0]     # add start token
# sentences

In [ ]:
final = []   # list of sentences, where sentences are a list of (word, POS)
all_words = []   # list of all words
for sentence in sentences:
  if len(sentence) == 0:   # if length of sentence is 0, discard
    continue
  words = sentence.split()   # split sentence into words
  all_words.extend(words)   
  word_tag_pair = []
  for word in words:
    word = word.split('_')   # split word and POS tag
    try:
      word_tag_pair.append((word[0], word[1]))   # append tupple
    except IndexError:
      pass
  final.append(word_tag_pair)   # tuples of a sentence to list
# final
len(all_words)

1216346

In [ ]:
sentence_lengths = []   # list containing length of all sentences
for sentence in sentences:
  sentence_lengths.append(len(sentence))    # add sentence length
print('max sentence length', max(sentence_lengths))
print('min sentence length', min(sentence_lengths))
print('avg sentence length', mean(sentence_lengths))
print('total number of sentences', len(sentence_lengths))

max sentence length 1968
min sentence length 11
avg sentence length 186.82925015867258
total number of sentences 55145


In [ ]:
vocab_word_tag = {}      # dictionary with word as key and all possible tags
tag_word_dict = {}      # dictionary with tag as key and all possible tags
for sentence in final:
  for pair in sentence:
    word = pair[0].lower()    # lowercase word
    tag = pair[1]             # get tag
    if word in vocab_word_tag:
      vocab_word_tag[word].add(tag)     # add tag in a set for a word
    else:
      vocab_word_tag[word] = set()
      vocab_word_tag[word].add(tag)      # add tag in a set for a word
    if tag in tag_word_dict:
      tag_word_dict[tag].add(word)      # add word in a set for a tag
    else:
      tag_word_dict[tag] = set()
      tag_word_dict[tag].add(word)      # add word in a set for a tag
print('word key tag value', len(vocab_word_tag))
print('tag key word value', len(tag_word_dict))

word key tag value 49810
tag key word value 473


In [ ]:
vocab = list(vocab_word_tag.keys())    # list of unique words -> vocab
vocab.sort()
len(vocab)
# vocab

49810

In [ ]:
full_tags = list(tag_word_dict.keys())     # list of unique tags
full_tags.sort()
len(full_tags)
# tags

473

In [ ]:
def get_tag_vocab(input_array):
  vocab_word_tag = {}        # dictionary with word as key and all possible tags
  tag_word_dict = {}         # dictionary with tag as key and all possible tags
  for sentence in input_array:
    for pair in sentence:
      word = pair[0].lower()    # lowercase word
      tag = pair[1]             # get tag
      if word in vocab_word_tag:
        vocab_word_tag[word].add(tag)    # add tag in a set for a word
      else:
        vocab_word_tag[word] = set()
        vocab_word_tag[word].add(tag)     # add tag in a set for a word
      if tag in tag_word_dict:
        tag_word_dict[tag].add(word)       # add word in a set for a tag
      else:
        tag_word_dict[tag] = set()
        tag_word_dict[tag].add(word)      # add word in a set for a tag
  vocab = list(vocab_word_tag.keys())     # list of unique words -> vocab
  vocab.append('<unknown>')       # add unknown token for unseen words in test data
  vocab.sort()
  tags = list(tag_word_dict.keys())        # list of unique tags
  tags.sort()
  vocab_word_tag['<unknown>'] = set()    
  for elt in tags:
    vocab_word_tag['<unknown>'].add(elt)    # add all possible tags with the unknown token
  return vocab, tags, vocab_word_tag

In [ ]:
num_words_for_tag = {}     # number of words for each tag
for key, val in tag_word_dict.items():
  num_words_for_tag[key] = len(val)
# num_words_for_tag

In [ ]:
total_words_vocab = vocab_word_tag.keys()    # total number of words in vocab
len(total_words_vocab)

49810

In [ ]:
def get_counts(input_array):
  transition_counts = {}     # transition counts: current tag previous tag
  emission_counts = {}       # emission counts: current word cuurent tag
  tag_counts = {}            # count of all tags
  for sentence in input_array:      # for each sentence
    prev_tag = '<s>'         # add start token
    for pair in sentence:
      word = pair[0].lower()   # lowercase word
      tag = pair[1]
      if (tag, prev_tag) not in transition_counts:    # increment for occurence of tag, prev tag
        transition_counts[(tag, prev_tag)] = 1
      else:
        transition_counts[(tag, prev_tag)] += 1        # increment for occurence of tag, prev tag
      if (word, tag) not in emission_counts:
        emission_counts[(word, tag)] = 1             # increment for occurence of word, tag
      else:
        emission_counts[(word, tag)] += 1            # increment for occurence of word, tag
      if tag not in tag_counts:
        tag_counts[tag] = 1       # increase tag count
      else:
        tag_counts[tag] += 1      # increase tag count
      prev_tag = tag  
  return transition_counts, emission_counts, tag_counts     # return dictionaries

In [ ]:
def get_transition_matrix(transition_counts, tag_counts, tags):
  transition_prob = {}      # transition probabilities: current tag previous tag
  for key, val in transition_counts.items():
    prev_tag = key[1]
    transition_prob[key] = val/tag_counts[prev_tag]    # count of current tag given previous tag / count of previous tag
  return transition_prob

In [ ]:
def get_emission_matrix(emission_counts, tag_counts, tags, vocab):
  emission_prob = {}       # emission probabilities: current word, current tag
  total_tag_count = sum(list(tag_counts.values()))
  for key, val in emission_counts.items():
    tag = key[1]
    emission_prob[key] = val/tag_counts[tag]    # count of current word given current tag / count of current tag
  for key, val in tag_counts.items():
    word = '<unknown>'
    emission_prob[(word, key)] = val/total_tag_count    # assign emission probability of unknown token using tag frequency
  return emission_prob

In [ ]:
def viterbi(transition_probs, emission_probs, test, vocab_word_tag):
  prev_word = '<s>'     # start tag
  predicted = []        # predicted tags
  count = 1
  for word in test:
    prob = []
    prob_tag = []
    for prev_tag in vocab_word_tag[prev_word]:     # consider all possible tags for previous word based on train data
      try:
        tag_list = vocab_word_tag[word]       # get all possible tags of current word based on train data
      except KeyError:
        tag_list = vocab_word_tag['<unknown>']    # if word not in train set vocab then assign unknown token
        word = '<unknown>'
      for cur_tag in tag_list:
        try:
          if word == '<unknown>':
            prob.append(np.log(transition_probs[(cur_tag, prev_tag)]))     # if word unknown only take transition probability
          else:
            prob.append(np.log(transition_probs[(cur_tag, prev_tag)]) + np.log(emission_probs[(word, cur_tag)]))   # otherwise sum of tranisition and emission probability
        except KeyError:
          prob.append(np.log(emission_probs[(word, cur_tag)]))   # if cur, prev tag combo not present consider only emission probability
        prob_tag.append(cur_tag)
      prev_word = word
    n = len(prob)
    max_prob = float('-inf')
    max_prob_tag = ''
    for i in range(n):
      if prob[i] > max_prob:     # find max probability tag
        max_prob = prob[i]
        max_prob_tag = prob_tag[i]
    splitted = max_prob_tag.split('-')
    if splitted[0] == 'FW':
      predicted.append(splitted[1]) 
    else: 
      predicted.append(splitted[0])
    # print(count, max_prob_tag)
    count += 1
  return predicted

In [ ]:
def word_tag_seperate(word_tag_pair_sentences):
  words = []
  tags = []
  for sentence in word_tag_pair_sentences:    # separate words and tags in sentences
    sentence_words = []
    sentences_tags = []
    for pair in sentence:
      words.append(pair[0].lower())    # lowercase word
      tags.append(pair[1])
  return words, tags

In [ ]:
kFold = KFold(n_splits=3)
predicted_tags = []      # predicted tags
all_test_tags = []       # actual tags
for train, test in kFold.split(final):    # 3 fold split
  training_set = []
  testing_set = []
  for elt in train:
    training_set.append(final[elt])     # get train set
  for elt in test:
    testing_set.append(final[elt])      # get test set
  test_words, test_tags = word_tag_seperate(testing_set)      # separate lists for test words and tags
  all_test_tags.append(test_tags)
  vocab, tags, vocab_word_tag = get_tag_vocab(training_set)    # get train vocab, tags, dictionary
  print(len(test_words))

  transition_counts, emission_counts, tag_counts = get_counts(training_set)    # transition, emission, tag counts from train set
  # print(transition_counts)
  # print(emission_counts)
  transition_probs = get_transition_matrix(transition_counts, tag_counts, tags)   # transition probability from train set
  emission_probs = get_emission_matrix(emission_counts, tag_counts, tags, vocab)  # emission probability from train set
  # print(transition_probs)
  predicted_tags.append(viterbi(transition_probs, emission_probs, test_words, vocab_word_tag))   # viterbi algorithm
  

446378
477594
292365


In [ ]:
def accuracy(actual, pred):   # calculate accuracy
  n = len(actual)
  correct = 0
  for i in range(n):
    if actual[i] == pred[i]:    # if actual=pred increment
      correct += 1
    # else:
    #   print(actual[i], pred[i])
  return correct/n     # correct/total

In [ ]:
for i in range(3):
  print(accuracy(all_test_tags[i], predicted_tags[i]))   
  # break

0.8076047654678322
0.8475944002646599
0.834976826911566


In [ ]:
def performance_metrics(actual, pred, tags, num):
  # print(len(actual), len(pred), len(tags))
  df = pd.DataFrame(0, columns = tags, index = tags) # columns are true tags, rows are predicted tags
  # print(df)
  n = len(actual)  
  for i in range(n):
    try:
      df[actual[i]][pred[i]] += 1      # increment count in confusion matrix
    except KeyError:
      pass
  df.to_csv(str(num) + '_confusion_matrix.csv')    # save to csv
  precision = {}
  recall = {}
  predicted_sum = df.sum(axis=0)     # calculate sum of predicted tags
  actual_sum = df.sum(axis=1)        # calculate sum of actual tags
  for tag in tags:
    precision[tag] = [df[tag][tag]/predicted_sum[tag]]     # tagwise precision
    recall[tag] = [df[tag][tag]/actual_sum[tag]]           # tagwise recall
  f1 = {}
  for tag in tags:
    f1[tag] = [harmonic_mean([precision[tag][0], recall[tag][0]])]    # F1 score is HM of precision and recall
  # print('Tagwise precision', precision)
  # print('Tagwise recall', recall)
  # print('Tagwise F1 score', f1)
  df_precision = pd.DataFrame.from_dict(precision) 
  df_recall = pd.DataFrame.from_dict(recall) 
  df_f1 = pd.DataFrame.from_dict(f1) 
  df_precision.to_csv(str(num) + '_precision.csv')
  df_recall.to_csv(str(num) + '_recall.csv')
  df_f1.to_csv(str(num) + '_f1.csv')

  overall_precision = df_precision.stack().mean()     # overall precision is average of tagwise
  overall_recall = df_recall.stack().mean()           # overall recall is average of tagwise
  overall_f1 = df_f1.stack().mean()                   # overall F1 score is average of tagwise

  print('Precision', overall_precision)
  print('Recall', overall_recall)
  print('F1 score', overall_f1)

In [ ]:
for i in range(3):
  print('Fold', i+1)     # calculate for each fold
  performance_metrics(all_test_tags[i], predicted_tags[i], full_tags, i)  

Fold 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in long_scalars
/usr/lib/python3.6/statistics.py:356: RuntimeWarning: divide by zero encountered in double_scalars
  T, total, count = _sum(1/x for x in _fail_neg(data, errmsg))


Precision 0.32152787909545194
Recall 0.7389588927962093
F1 score 0.757546302812592
Fold 2
Precision 0.2747432775971587
Recall 0.7375803646944107
F1 score 0.7902254630792261
Fold 3
Precision 0.3998570743977475
Recall 0.7575858149496483
F1 score 0.7536152170715897
